In [ ]:
import sys
import os

notebook_path = os.getcwd() 
parent_dir = os.path.dirname(notebook_path)
project_root = os.path.dirname(parent_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from copy import deepcopy

In [3]:
# import state_NN_models
import Filters
import utils
import Systems
from utils import losses, trainer, utils
from torch.utils.data import TensorDataset, DataLoader, random_split
from state_NN_models.StateBayesianKalmanNet import StateBayesianKalmanNet
from state_NN_models.StateKalmanNet import StateKalmanNet
from state_NN_models.RecursiveKalmanNet import StateRecursiveKalmanNet

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Používané zařízení: {device}")

Používané zařízení: cuda


In [5]:
state_dim_2d = 2
obs_dim_2d = 2

F_base_2d = torch.tensor([[1.0, 1.0], 
                          [0.0, 1.0]])

svd_F = torch.linalg.svd(F_base_2d)
F_true_2d = F_base_2d / svd_F.S[0]
print(F_true_2d)

H_true_2d = torch.eye(obs_dim_2d)

Q_true_2d = torch.eye(state_dim_2d) * 0.5 # Šum procesu
R_true_2d = torch.eye(obs_dim_2d) * 0.1 # Šum měření

# Počáteční podmínky
Ex0_true_2d = torch.tensor([[1.0], [0.0]])
P0_true_2d = torch.eye(state_dim_2d) * 1.5
F_model_2d = F_true_2d
H_model_2d = H_true_2d
Q_model_2d = torch.eye(state_dim_2d) * 0.1
R_model_2d = R_true_2d
Ex0_model_2d = torch.tensor([[0.5], [0.5]])
P0_model_2d = torch.eye(state_dim_2d) * 1.0

print("\nInicializuji 2D Linear_Canonical systém (replikace autorů)...")
sys_true = Systems.DynamicSystem(
    state_dim=state_dim_2d, obs_dim=obs_dim_2d,
    Ex0=Ex0_true_2d, P0=P0_true_2d,
    Q=Q_true_2d, R=R_true_2d,
    F=F_true_2d, H=H_true_2d,
    device=device
)
sys_model = Systems.DynamicSystem(
    state_dim=state_dim_2d, obs_dim=obs_dim_2d,
    Ex0=Ex0_model_2d, P0=P0_model_2d,
    Q=Q_model_2d, R=R_model_2d,
    F=F_model_2d, H=H_model_2d,
    device=device
)
print("... 2D systém inicializován.")

tensor([[0.6180, 0.6180],
        [0.0000, 0.6180]])

Inicializuji 2D Linear_Canonical systém (replikace autorů)...
... 2D systém inicializován.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from copy import deepcopy

def train_RKN(
    model, 
    train_loader, 
    val_loader, 
    device, 
    epochs=100, 
    lr=1e-3, 
    clip_grad=1.0, 
    early_stopping_patience=20, 
    optimizer_type=torch.optim.AdamW,
    weight_decay=1e-5,
    use_nll_loss=False
):
    if use_nll_loss and not model.returns_covariance:
        raise ValueError("Cannot use NLL Loss if model.returns_covariance is False.")

    optimizer = optimizer_type(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

    best_val_loss = float('inf')
    best_val_mse_at_opt = float('inf')
    optimal_epoch = 0
    epochs_no_improve = 0
    best_model_state = None

    print(f"🚀 START RKN Training (Epochs: {epochs}, LR: {lr}, NLL Loss: {use_nll_loss})")

    for epoch in range(epochs):
        model.train()
        train_loss_sum = 0.0
        train_mse_metric_sum = 0.0

        for x_true_batch, y_meas_batch in train_loader:
            x_true_batch = x_true_batch.to(device)
            y_meas_batch = y_meas_batch.to(device)
            batch_size, seq_len, _ = x_true_batch.shape

            optimizer.zero_grad()
            model.reset(batch_size=batch_size, initial_state=x_true_batch[:, 0, :])

            predictions_x = []
            predictions_P = []

            for t in range(1, seq_len):
                y_t = y_meas_batch[:, t, :]
                
                if model.returns_covariance:
                    x_filtered_t, P_filtered_t, _ = model.step(y_t)
                    predictions_P.append(P_filtered_t)
                else:
                    x_filtered_t, _ = model.step(y_t)
                predictions_x.append(x_filtered_t)

            predicted_trajectory = torch.stack(predictions_x, dim=1)
            target_trajectory = x_true_batch[:, 1:, :]

            if use_nll_loss:
                predicted_cov_trajectory = torch.stack(predictions_P, dim=1)
                I_eps = torch.eye(model.state_dim, device=device).view(1, 1, model.state_dim, model.state_dim) * 1e-6
                safe_cov = predicted_cov_trajectory + I_eps
                
                error = (target_trajectory - predicted_trajectory).unsqueeze(-1)
                flat_cov = safe_cov.view(-1, model.state_dim, model.state_dim)
                flat_err = error.view(-1, model.state_dim, 1)
                
                try:
                    cov_inv = torch.linalg.pinv(flat_cov, hermitian=True)
                    mahalanobis = torch.bmm(torch.bmm(flat_err.transpose(1, 2), cov_inv), flat_err).squeeze()
                    sign, logdet = torch.linalg.slogdet(flat_cov)
                    loss = 0.5 * (mahalanobis + logdet).mean()
                except torch._C._LinAlgError:
                    loss = F.mse_loss(predicted_trajectory, target_trajectory)
            else:
                loss = F.mse_loss(predicted_trajectory, target_trajectory)

            loss.backward()
            if clip_grad > 0:
                nn.utils.clip_grad_norm_(model.parameters(), clip_grad)
            optimizer.step()
            
            train_loss_sum += loss.item()
            with torch.no_grad():
                train_mse_metric_sum += F.mse_loss(predicted_trajectory, target_trajectory).item()

        scheduler.step()
        avg_train_loss = train_loss_sum / len(train_loader)
        avg_train_mse = train_mse_metric_sum / len(train_loader)

        # === Validation Phase ===
        model.eval()
        val_loss_sum = 0.0
        val_mse_sum = 0.0
        val_anees_sum = 0.0  # <--- Sběr ANEES
        
        with torch.no_grad():
            for x_true_val, y_meas_val in val_loader:
                x_true_val = x_true_val.to(device)
                y_meas_val = y_meas_val.to(device)
                batch_size_val, seq_len_val, _ = x_true_val.shape

                model.reset(batch_size=batch_size_val, initial_state=x_true_val[:, 0, :])
                
                val_predictions_x = []
                val_predictions_P = []

                for t in range(1, seq_len_val):
                    y_t_val = y_meas_val[:, t, :]
                    if model.returns_covariance:
                        x_filtered_t_val, P_filtered_t_val, _ = model.step(y_t_val)
                        val_predictions_P.append(P_filtered_t_val)
                    else:
                        x_filtered_t_val, _ = model.step(y_t_val)
                    val_predictions_x.append(x_filtered_t_val)
                    
                predicted_val_trajectory = torch.stack(val_predictions_x, dim=1)
                target_val_trajectory = x_true_val[:, 1:, :]

                if use_nll_loss:
                    predicted_cov_val = torch.stack(val_predictions_P, dim=1)
                    safe_cov_val = predicted_cov_val + I_eps
                    error_val = (target_val_trajectory - predicted_val_trajectory).unsqueeze(-1)
                    flat_cov_v = safe_cov_val.view(-1, model.state_dim, model.state_dim)
                    flat_err_v = error_val.view(-1, model.state_dim, 1)
                    
                    try:
                        cov_inv_v = torch.linalg.pinv(flat_cov_v, hermitian=True)
                        mahal_v = torch.bmm(torch.bmm(flat_err_v.transpose(1, 2), cov_inv_v), flat_err_v).squeeze()
                        _, logdet_v = torch.linalg.slogdet(flat_cov_v)
                        loss_v = 0.5 * (mahal_v + logdet_v).mean()
                        
                        val_anees_sum += mahal_v.mean().item()  # <--- Ukládáme průměrný ANEES za dávku
                    except torch._C._LinAlgError:
                        loss_v = F.mse_loss(predicted_val_trajectory, target_val_trajectory)
                        val_anees_sum += float('nan')
                else:
                    loss_v = F.mse_loss(predicted_val_trajectory, target_val_trajectory)
                
                val_loss_sum += loss_v.item()
                val_mse_sum += F.mse_loss(predicted_val_trajectory, target_val_trajectory).item()

        avg_val_loss = val_loss_sum / len(val_loader)
        avg_val_mse = val_mse_sum / len(val_loader)
        avg_val_anees = val_anees_sum / len(val_loader) if use_nll_loss else float('nan')
        
        # --- Nový logovací formát s ANEES ---
        if use_nll_loss:
            print(f"Epoch: {epoch+1}/{epochs} | Train NLL: {avg_train_loss:.4f} | Val NLL: {avg_val_loss:.4f} | MSE: {avg_val_mse:.4f} | ANEES: {avg_val_anees:.2f} | LR: {scheduler.get_last_lr()[0]:.2e}")
        else:
            print(f"Epoch: {epoch+1}/{epochs} | Train MSE: {avg_train_loss:.4f} | Val MSE: {avg_val_loss:.4f} | LR: {scheduler.get_last_lr()[0]:.2e}")
        
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_val_mse_at_opt = avg_val_mse
            optimal_epoch = epoch + 1
            epochs_no_improve = 0
            best_model_state = deepcopy(model.state_dict())
            print(f"  >>> Optimal idx: {optimal_epoch} | Optimal Val Loss: {best_val_loss:.4f} <<<")
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= early_stopping_patience:
            print(f"\nEarly stopping triggered after {epoch + 1} epochs.")
            break
            
    print("\nTraining complete. Best Validation Loss:", best_val_loss)
    if best_model_state:
        model.load_state_dict(best_model_state)
        print("Optimal model loaded.")
        
    return {
        "final_model": model,
        "best_epoch": optimal_epoch,
        "best_val_loss": best_val_loss,
        "best_val_mse": best_val_mse_at_opt
    }

In [7]:
TRAIN_SEQ_LEN = 10      # Krátké sekvence pro stabilní trénink (TBPTT)
VALID_SEQ_LEN = 20      # Stejná délka pro konzistentní validaci
TEST_SEQ_LEN = 100      # Dlouhé sekvence pro testování generalizace

NUM_TRAIN_TRAJ = 500   # Hodně trénovacích příkladů
NUM_VALID_TRAJ = 200    # Dostatek pro spolehlivou validaci
NUM_TEST_TRAJ = 100     # Pro robustní vyhodnocení

BATCH_SIZE = 8         # Dobrý kompromis

x_train, y_train = utils.generate_data(sys_true, num_trajectories=NUM_TRAIN_TRAJ, seq_len=TRAIN_SEQ_LEN)
x_val, y_val = utils.generate_data(sys_true, num_trajectories=NUM_VALID_TRAJ, seq_len=VALID_SEQ_LEN)
x_test, y_test = utils.generate_data(sys_true, num_trajectories=1, seq_len=TEST_SEQ_LEN)

train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [13]:
import random
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
rkn_architecture_config = {
    'nb_layer_FC1': 1,
    'FC1_mult': 10,
    'nbr_GRU': 1,
    'hidden_size_mult': 10,
    'nb_layer_FC2': 1,
    'FC2_mult': 20,
}

# Parametry pro konstruktor StateRecursiveKalmanNet
model_config = {
    "config": rkn_architecture_config,
    "weight_factor": 0.1,         # Autoři používají pro zmenšení iniciálních vah
    "returns_covariance": True    # Povolí výpočet matice P (nutné, pokud chceme zkusit NLL Loss)
}

# Parametry pro trénovací funkci train_RKN
train_config = {
    "epochs": 100,               # RKN zpracovává sekvence celostně, používáme epochy
    "lr": 1e-3,
    "clip_grad": 10.0,
    "early_stopping_patience": 20,
    "use_nll_loss": True        # Zpočátku doporučuji False (čisté MSE) pro maximální stabilitu
}

print("="*80)
print("Spouštím jeden plnohodnotný tréninkový běh (Recursive KalmanNet)...")
print(f"Parametry architektury: {rkn_architecture_config}")
print(f"Parametry tréninku: {train_config}")
print("="*80)

# Nastavení seedu
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Vytvoření modelu (Předpokládám, že třída StateRecursiveKalmanNet je spuštěna v buňce nad tímto)
state_rkn = StateRecursiveKalmanNet(
    sys_model,
    device=device,
    **model_config
).to(device)

# Spuštění tréninku pomocí nové funkce
results = train_RKN(
    model=state_rkn,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    **train_config
)

trained_model = results['final_model']

print("\n" + "="*80)
print("TRÉNINK DOKONČEN - FINÁLNÍ VÝSLEDKY Z NEJLEPŠÍHO MODELU")
print("="*80)
print(f"Nejlepší model byl nalezen v epoše: {results['best_epoch']}")
print("--- Metriky odpovídající tomuto nejlepšímu modelu ---")
print(f"  MSE na validační sadě:       {results['best_val_mse']:.4f}")

# NLL a ANEES vypisujeme pouze pokud jsme optimalizovali s NLL
if train_config['use_nll_loss']:
    print(f"  Validační Loss (NLL):        {results['best_val_loss']:.4f}")
else:
    print(f"  Validační Loss (MSE):        {results['best_val_loss']:.4f}")
print("="*80)

Spouštím jeden plnohodnotný tréninkový běh (Recursive KalmanNet)...
Parametry architektury: {'nb_layer_FC1': 1, 'FC1_mult': 10, 'nbr_GRU': 1, 'hidden_size_mult': 10, 'nb_layer_FC2': 1, 'FC2_mult': 20}
Parametry tréninku: {'epochs': 100, 'lr': 0.001, 'clip_grad': 10.0, 'early_stopping_patience': 20, 'use_nll_loss': True}
🚀 START RKN Training (Epochs: 100, LR: 0.001, NLL Loss: True)
Epoch: 1/100 | Train Loss: 103.5805 | Val Loss: -1.3828 (MSE: 0.0907) | LR: 1.00e-03
  >>> Optimal idx: 1 | Optimal Val Loss: -1.3828 <<<
Epoch: 2/100 | Train Loss: -1.4447 | Val Loss: -1.4508 (MSE: 0.0853) | LR: 1.00e-03
  >>> Optimal idx: 2 | Optimal Val Loss: -1.4508 <<<
Epoch: 3/100 | Train Loss: -1.4589 | Val Loss: -1.4392 (MSE: 0.0862) | LR: 1.00e-03
Epoch: 4/100 | Train Loss: -1.4630 | Val Loss: -1.4588 (MSE: 0.0852) | LR: 1.00e-03
  >>> Optimal idx: 4 | Optimal Val Loss: -1.4588 <<<
Epoch: 5/100 | Train Loss: -1.4607 | Val Loss: -1.4510 (MSE: 0.0856) | LR: 1.00e-03
Epoch: 6/100 | Train Loss: -1.4601 |

In [9]:

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import os
import random
import csv
from datetime import datetime
import pandas as pd
from copy import deepcopy
# Nastavení seedu pro reprodukovatelnost tohoto běhu
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
state_knet = StateKalmanNet(sys_model, device=device, hidden_size_multiplier=12).to(device)
trainer.train_state_KalmanNet(
    model=state_knet, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    device=device, 
    epochs=100, 
    lr=1e-4,
    early_stopping_patience=30
)

DEBUG: Layer 'output_final_linear.0' initialized near zero (Start K=0).
New best model saved! Epoch [1/100], Train Loss: 0.704641, Val Loss: 0.585440
New best model saved! Epoch [2/100], Train Loss: 0.475780, Val Loss: 0.391406
New best model saved! Epoch [3/100], Train Loss: 0.338607, Val Loss: 0.287173
New best model saved! Epoch [4/100], Train Loss: 0.254932, Val Loss: 0.218205
Epoch [5/100] | Train Loss: 0.1936 (Pos: 0.00, Vel: 0.00) | Val Loss: 0.1633 | Val MSE: 0.16
Epoch [4/100], Train Loss: 0.254932, Val Loss: 0.218205
New best model saved! Epoch [5/100], Train Loss: 0.193583, Val Loss: 0.163264
New best model saved! Epoch [6/100], Train Loss: 0.143375, Val Loss: 0.120105
New best model saved! Epoch [7/100], Train Loss: 0.110473, Val Loss: 0.099017
New best model saved! Epoch [8/100], Train Loss: 0.095222, Val Loss: 0.090171
New best model saved! Epoch [9/100], Train Loss: 0.088264, Val Loss: 0.086727
Epoch [10/100] | Train Loss: 0.0857 (Pos: 0.00, Vel: 0.00) | Val Loss: 0.0855

StateKalmanNet(
  (dnn): DNN_KalmanNet(
    (input_layer): Sequential(
      (0): Linear(in_features=8, out_features=384, bias=True)
      (1): ReLU()
    )
    (gru): GRU(384, 8)
    (output_hidden_layer): Sequential(
      (0): Linear(in_features=8, out_features=16, bias=True)
      (1): ReLU()
    )
    (output_final_linear): Sequential(
      (0): Linear(in_features=16, out_features=4, bias=True)
    )
  )
)

In [ ]:

# import torch
# import torch.nn as nn
# from torch.utils.data import TensorDataset, DataLoader
# import numpy as np
# import os
# import random
# import csv
# from datetime import datetime
# import pandas as pd
# from copy import deepcopy
# from state_NN_models.StateKalmanNetWithKnownR import StateKalmanNetWithKnownR
# # Nastavení seedu pro reprodukovatelnost tohoto běhu
# torch.manual_seed(42)
# np.random.seed(42)
# random.seed(42)
# state_knetR = StateKalmanNetWithKnownR(sys_model, device=device, hidden_size_multiplier=12).to(device)
# trainer.train_state_KalmanNet(
#     model=state_knetR, 
#     train_loader=train_loader, 
#     val_loader=val_loader, 
#     device=device, 
#     epochs=50, 
#     lr=1e-4,
#     early_stopping_patience=30
# )

TypeError: DNN_KalmanNet.forward() missing 3 required positional arguments: 'diff_state', 'diff_obs', and 'h_prev'

In [14]:
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

# ==============================================================================
# 0. PŘEDPOKLADY - ZDE PŘIŘAĎTE VAŠE NATRÉNOVANÉ MODELY
# ==============================================================================
# Ujistěte se, že v proměnných níže máte již natrénované a připravené modely.
try:
    trained_model_rkn = trained_model  # Změněno z BKN na RKN
    trained_model_classic = state_knet
    # trained_model_knetR = state_knetR
    print("INFO: Všechny natrénované modely nalezeny a přiřazeny.")
except NameError:
    print("VAROVÁNÍ: Některé z proměnných `trained_model`, `state_knet`, nebo `state_knetR` nebyly nalezeny.")
    print("         Ujistěte se, že jste nejprve úspěšně dokončili trénink všech modelů.")


# ==============================================================================
# 1. KONFIGURACE TESTU
# ==============================================================================
TEST_SEQ_LEN = 1000
NUM_TEST_TRAJ = 10
# J_SAMPLES_TEST = 25  <-- Odstraněno, RKN je deterministický a nevyžaduje vzorkování

# ==============================================================================
# 2. PŘÍPRAVA DAT
# ==============================================================================
print(f"\nGeneruji {NUM_TEST_TRAJ} testovacích trajektorií o délce {TEST_SEQ_LEN}...")
x_test, y_test = utils.generate_data(sys_true, num_trajectories=NUM_TEST_TRAJ, seq_len=TEST_SEQ_LEN)
test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
print("Generování dat dokončeno.")

# ==============================================================================
# 3. INICIALIZACE VŠECH FILTRŮ PRO POROVNÁNÍ
# ==============================================================================
ekf_mismatched = Filters.ExtendedKalmanFilter(sys_model)
ekf_ideal = Filters.ExtendedKalmanFilter(sys_true)
ukf_mismatched = Filters.UnscentedKalmanFilter(sys_model)
ukf_ideal = Filters.UnscentedKalmanFilter(sys_true)
akf_mismatched = Filters.AdaptiveKalmanFilter(sys_model,mdm_L=3,mdm_version=2)
kf_ideal = Filters.KalmanFilter(sys_true)
print("Všechny model-based filtry (EKF, UKF, AKF) inicializovány.")

# ==============================================================================
# 4. VYHODNOCOVACÍ SMYČKA
# ==============================================================================
# Seznamy pro ukládání výsledků z každé trajektorie
all_x_true_cpu = []
all_x_hat_rkn_cpu, all_P_hat_rkn_cpu = [], []  # Přejmenováno na RKN
all_x_hat_classic_knet_cpu = []
# all_x_hat_knetR_cpu, all_P_hat_knetR_cpu = [], []
all_x_hat_ekf_mismatched_cpu, all_P_hat_ekf_mismatched_cpu = [], []
all_x_hat_ekf_ideal_cpu, all_P_hat_ekf_ideal_cpu = [], []
all_x_hat_ukf_mismatched_cpu, all_P_hat_ukf_mismatched_cpu = [], []
all_x_hat_ukf_ideal_cpu, all_P_hat_ukf_ideal_cpu = [], []
all_x_hat_akf_mismatched_cpu, all_P_hat_akf_mismatched_cpu = [], []
all_x_hat_kf_ideal_cpu, all_P_hat_kf_ideal_cpu = [], []

print(f"\nVyhodnocuji modely na {NUM_TEST_TRAJ} testovacích trajektoriích...")

trained_model_rkn.eval() 
trained_model_classic.eval()
# trained_model_knetR.eval()

with torch.no_grad():
    for i, (x_true_seq_batch, y_test_seq_batch) in enumerate(test_loader):
        y_test_seq_gpu = y_test_seq_batch.squeeze(0).to(device)
        x_true_seq_gpu = x_true_seq_batch.squeeze(0).to(device)
        initial_state = x_true_seq_gpu[0, :].unsqueeze(0)
        
        # --- A. Recursive KalmanNet (RKN) ---
        trained_model_rkn.reset(batch_size=1, initial_state=initial_state)
        rkn_preds_x, rkn_preds_P = [], []
        
        for t in range(1, TEST_SEQ_LEN):
            # Model RKN vrací (x_filtered, P_filtered, reg_loss)
            step_output = trained_model_rkn.step(y_test_seq_gpu[t, :].unsqueeze(0))
            x_filtered_t = step_output[0]
            P_filtered_t = step_output[1]
            
            rkn_preds_x.append(x_filtered_t)
            rkn_preds_P.append(P_filtered_t)
            
        full_x_hat_rkn = torch.cat([initial_state, torch.cat(rkn_preds_x, dim=0)], dim=0)
        
        # Ošetření dimenzí pro matici P z RKN
        processed_P_rkn_list = []
        for p_tensor in rkn_preds_P:
            while p_tensor.dim() < 2:
                p_tensor = p_tensor.unsqueeze(-1)
            if p_tensor.dim() > 2 and p_tensor.shape[0] == 1:
                p_tensor = p_tensor.squeeze(0)
            processed_P_rkn_list.append(p_tensor)

        P_sequence_rkn = torch.stack(processed_P_rkn_list, dim=0)
        
        P0_for_cat_rkn = sys_model.P0.clone()
        while P0_for_cat_rkn.dim() < P_sequence_rkn.dim():
            P0_for_cat_rkn = P0_for_cat_rkn.unsqueeze(0)
            
        full_P_hat_rkn = torch.cat([P0_for_cat_rkn, P_sequence_rkn], dim=0)

        # --- B. Klasický StateKalmanNet (pouze MSE) ---
        trained_model_classic.reset(batch_size=1, initial_state=initial_state)
        classic_knet_preds = []
        for t in range(1, TEST_SEQ_LEN):
            x_filtered_t = trained_model_classic.step(y_test_seq_gpu[t, :].unsqueeze(0))
            classic_knet_preds.append(x_filtered_t)
        full_x_hat_classic_knet = torch.cat([initial_state, torch.cat(classic_knet_preds, dim=0)], dim=0)
        
        # # --- C. StateKalmanNetWithKnownR ---
        # trained_model_knetR.reset(batch_size=1, initial_state=initial_state)
        # knetR_preds_x, knetR_preds_P = [], []
        # for t in range(1, TEST_SEQ_LEN):
        #     x_filtered_t, P_filtered_t = trained_model_knetR.step(y_test_seq_gpu[t, :].unsqueeze(0))
        #     knetR_preds_x.append(x_filtered_t)
        #     knetR_preds_P.append(P_filtered_t)
        # full_x_hat_knetR = torch.cat([initial_state, torch.cat(knetR_preds_x, dim=0)], dim=0)
        # processed_P_list = []
        # for p_tensor in knetR_preds_P:

        #     while p_tensor.dim() < 2:
        #         p_tensor = p_tensor.unsqueeze(-1)

        #     if p_tensor.dim() > 2 and p_tensor.shape[0] == 1:
        #         p_tensor = p_tensor.squeeze(0)
        #     processed_P_list.append(p_tensor)

        # P_sequence_knetR = torch.stack(processed_P_list, dim=0)
        
        # P0_for_cat = sys_model.P0.clone()
        # while P0_for_cat.dim() < P_sequence_knetR.dim():
        #     P0_for_cat = P0_for_cat.unsqueeze(0)
            
        # full_P_hat_knetR = torch.cat([P0_for_cat, P_sequence_knetR], dim=0)

        # --- D. EKF (nepřesný a ideální) ---
        ekf_m_res = ekf_mismatched.process_sequence(y_test_seq_gpu, Ex0=sys_model.Ex0, P0=sys_model.P0)
        full_x_hat_ekf_m = ekf_m_res['x_filtered']
        full_P_hat_ekf_m = ekf_m_res['P_filtered'] 

        ekf_i_res = ekf_ideal.process_sequence(y_test_seq_gpu, Ex0=sys_true.Ex0, P0=sys_true.P0)
        full_x_hat_ekf_i = ekf_i_res['x_filtered']
        full_P_hat_ekf_i = ekf_i_res['P_filtered']

        # --- E. UKF (nepřesný a ideální) ---
        ukf_m_res = ukf_mismatched.process_sequence(y_test_seq_gpu, Ex0=sys_model.Ex0, P0=sys_model.P0)
        full_x_hat_ukf_m = ukf_m_res['x_filtered']
        full_P_hat_ukf_m = ukf_m_res['P_filtered']

        ukf_i_res = ukf_ideal.process_sequence(y_test_seq_gpu, Ex0=sys_true.Ex0, P0=sys_true.P0)
        full_x_hat_ukf_i = ukf_i_res['x_filtered']
        full_P_hat_ukf_i = ukf_i_res['P_filtered']

        # --- F. Adaptivní EKF (nepřesný) ---
        akf_m_res,_,_ = akf_mismatched.process_sequence_adaptively(y_test_seq_gpu)
        full_x_hat_akf_m = akf_m_res['x_filtered']
        full_P_hat_akf_m = akf_m_res['P_filtered']

        kf_i_res = kf_ideal.process_sequence(y_test_seq_gpu, Ex0=sys_true.Ex0, P0=sys_true.P0)
        full_x_hat_kf_i = kf_i_res['x_filtered']
        full_P_hat_kf_i = kf_i_res['P_filtered']

        all_x_true_cpu.append(x_true_seq_gpu.cpu())
        all_x_hat_rkn_cpu.append(full_x_hat_rkn.cpu()); all_P_hat_rkn_cpu.append(full_P_hat_rkn.cpu())
        all_x_hat_classic_knet_cpu.append(full_x_hat_classic_knet.cpu())
        # all_x_hat_knetR_cpu.append(full_x_hat_knetR.cpu()); all_P_hat_knetR_cpu.append(full_P_hat_knetR.cpu())
        all_x_hat_ekf_mismatched_cpu.append(full_x_hat_ekf_m.cpu()); all_P_hat_ekf_mismatched_cpu.append(full_P_hat_ekf_m.cpu())
        all_x_hat_ekf_ideal_cpu.append(full_x_hat_ekf_i.cpu()); all_P_hat_ekf_ideal_cpu.append(full_P_hat_ekf_i.cpu())
        all_x_hat_ukf_mismatched_cpu.append(full_x_hat_ukf_m.cpu()); all_P_hat_ukf_mismatched_cpu.append(full_P_hat_ukf_m.cpu())
        all_x_hat_ukf_ideal_cpu.append(full_x_hat_ukf_i.cpu()); all_P_hat_ukf_ideal_cpu.append(full_P_hat_ukf_i.cpu())
        all_x_hat_akf_mismatched_cpu.append(full_x_hat_akf_m.cpu()); all_P_hat_akf_mismatched_cpu.append(full_P_hat_akf_m.cpu())
        all_x_hat_kf_ideal_cpu.append(full_x_hat_kf_i.cpu()); all_P_hat_kf_ideal_cpu.append(full_P_hat_kf_i.cpu())

        print(f"Dokončena trajektorie {i + 1}/{NUM_TEST_TRAJ}...")

# ==============================================================================
# 5. FINÁLNÍ VÝPOČET A VÝPIS METRIK
# ==============================================================================
# Seznamy pro sběr metrik
mse_rkn, anees_rkn = [], []; mse_classic_knet = []; mse_knetR, anees_knetR = [], []
mse_ekf_mis, anees_ekf_mis = [], []; mse_ekf_ideal, anees_ekf_ideal = [], []
mse_ukf_mis, anees_ukf_mis = [], []; mse_ukf_ideal, anees_ukf_ideal = [], []
mse_akf_mis, anees_akf_mis = [], []; mse_kf_ideal, anees_kf_ideal = [], []

print("\nPočítám finální metriky pro jednotlivé trajektorie...")

with torch.no_grad():
    for i in range(NUM_TEST_TRAJ):
        x_true = all_x_true_cpu[i]
        def get_metrics(x_hat, P_hat):
            mse = F.mse_loss(x_hat[1:], x_true[1:]).item()
            anees = utils.calculate_anees_vectorized(x_true.unsqueeze(0), x_hat.unsqueeze(0), P_hat.unsqueeze(0))
            return mse, anees

        # Výpočty pro všechny modely
        mse, anees = get_metrics(all_x_hat_rkn_cpu[i], all_P_hat_rkn_cpu[i]); mse_rkn.append(mse); anees_rkn.append(anees)
        mse = F.mse_loss(all_x_hat_classic_knet_cpu[i][1:], x_true[1:]).item(); mse_classic_knet.append(mse)
        # mse, anees = get_metrics(all_x_hat_knetR_cpu[i], all_P_hat_knetR_cpu[i]); mse_knetR.append(mse); anees_knetR.append(anees)
        mse, anees = get_metrics(all_x_hat_ekf_mismatched_cpu[i], all_P_hat_ekf_mismatched_cpu[i]); mse_ekf_mis.append(mse); anees_ekf_mis.append(anees)
        mse, anees = get_metrics(all_x_hat_ekf_ideal_cpu[i], all_P_hat_ekf_ideal_cpu[i]); mse_ekf_ideal.append(mse); anees_ekf_ideal.append(anees)
        mse, anees = get_metrics(all_x_hat_ukf_mismatched_cpu[i], all_P_hat_ukf_mismatched_cpu[i]); mse_ukf_mis.append(mse); anees_ukf_mis.append(anees)
        mse, anees = get_metrics(all_x_hat_ukf_ideal_cpu[i], all_P_hat_ukf_ideal_cpu[i]); mse_ukf_ideal.append(mse); anees_ukf_ideal.append(anees)
        mse, anees = get_metrics(all_x_hat_akf_mismatched_cpu[i], all_P_hat_akf_mismatched_cpu[i]); mse_akf_mis.append(mse); anees_akf_mis.append(anees)
        mse, anees = get_metrics(all_x_hat_kf_ideal_cpu[i], all_P_hat_kf_ideal_cpu[i]); mse_kf_ideal.append(mse); anees_kf_ideal.append(anees)

# Funkce pro bezpečné průměrování
def avg(metric_list): return np.mean([m for m in metric_list if not np.isnan(m)])
state_dim_for_nees = all_x_true_cpu[0].shape[1]

# --- Finální výpis tabulky ---
print("\n" + "="*80)
print(f"FINÁLNÍ VÝSLEDKY (průměr přes {NUM_TEST_TRAJ} běhů)")
print("="*80)
print(f"{'Model':<35} | {'Průměrné MSE':<20} | {'Průměrný ANEES':<20}")
print("-" * 80)
print(f"{'--- Data-Driven Models ---':<35} | {'(nižší je lepší)':<20} | {'(bližší ' + str(float(state_dim_for_nees)) + ' je lepší)':<20}")
print(f"{'Recursive KNet (RKN)':<35} | {avg(mse_rkn):<20.4f} | {avg(anees_rkn):<20.4f}")
print(f"{'KNet (pouze MSE)':<35} | {avg(mse_classic_knet):<20.4f} | {'N/A':<20}")
# print(f"{'KNet with Known R (KNetR)':<35} | {avg(mse_knetR):<20.4f} | {avg(anees_knetR):<20.4f}")
print("-" * 80)
print(f"{'--- Model-Based Filters ---':<35} | {'':<20} | {'':<20}")
print(f"{'EKF (Nepřesný model)':<35} | {avg(mse_ekf_mis):<20.4f} | {avg(anees_ekf_mis):<20.4f}")
print(f"{'UKF (Nepřesný model)':<35} | {avg(mse_ukf_mis):<20.4f} | {avg(anees_ukf_mis):<20.4f}")
print(f"{'AKF (Nepřesný model)':<35} | {avg(mse_akf_mis):<20.4f} | {avg(anees_akf_mis):<20.4f}")
print("-" * 80)
print(f"{'--- Benchmarks ---':<35} | {'':<20} | {'':<20}")
print(f"{'EKF (Ideální model)':<35} | {avg(mse_ekf_ideal):<20.4f} | {avg(anees_ekf_ideal):<20.4f}")
print(f"{'UKF (Ideální model)':<35} | {avg(mse_ukf_ideal):<20.4f} | {avg(anees_ukf_ideal):<20.4f}")
print(f"{'KF (Ideální model)':<35} | {avg(mse_kf_ideal):<20.4f} | {avg(anees_kf_ideal):<20.4f}")
print("="*80)

INFO: Všechny natrénované modely nalezeny a přiřazeny.

Generuji 10 testovacích trajektorií o délce 1000...
Generování dat dokončeno.
Všechny model-based filtry (EKF, UKF, AKF) inicializovány.

Vyhodnocuji modely na 10 testovacích trajektoriích...
Spouštím MDM odhad Q a R...
MDM odhad dokončen.
Odhadnuté Q:
 [[0.53521981 0.00161313]
 [0.00161313 0.55229666]]
Odhadnuté R:
 [[0.07874503 0.00212589]
 [0.00212589 0.03761439]]
Matice Q a R v Kalmanově filtru byly aktualizovány.
Dokončena trajektorie 1/10...
Spouštím MDM odhad Q a R...
MDM odhad dokončen.
Odhadnuté Q:
 [[0.51748314 0.00151425]
 [0.00151425 0.50499735]]
Odhadnuté R:
 [[ 0.07691707 -0.00475298]
 [-0.00475298  0.12057272]]
Matice Q a R v Kalmanově filtru byly aktualizovány.
Dokončena trajektorie 2/10...
Spouštím MDM odhad Q a R...
MDM odhad dokončen.
Odhadnuté Q:
 [[ 0.54914774 -0.04203468]
 [-0.04203468  0.55944857]]
Odhadnuté R:
 [[0.06492597 0.02485289]
 [0.02485289 0.09486328]]
Matice Q a R v Kalmanově filtru byly aktualizo

# Kalman Gain comparison